## Class Imbalancing Implement on MNIST dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import numpy as np


### Load MNIST dataset

In [2]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

### Define the CNN model

In [3]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


### Train the CNN model on original dataset

In [4]:
# Train the CNN model on original dataset
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

def test_model(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    inputs, logits = [], []  # Correct initialization
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            # Store input vectors and logits
            inputs.append(data)
            logits.append(output)

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')
    return inputs, logits, accuracy

# Instantiate and train the model
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, criterion, optimizer)
inputs_original, logits_original, original_accuracy = test_model(model, test_loader)


Epoch 1/10, Loss: 0.003921981900930405
Epoch 2/10, Loss: 0.0023901760578155518
Epoch 3/10, Loss: 0.010086002759635448
Epoch 4/10, Loss: 0.0012183162616565824
Epoch 5/10, Loss: 0.03364719823002815
Epoch 6/10, Loss: 0.056069210171699524
Epoch 7/10, Loss: 0.00759118003770709
Epoch 8/10, Loss: 0.000558324798475951
Epoch 9/10, Loss: 0.00045810535084456205
Epoch 10/10, Loss: 1.6141477317432873e-05
Test set: Average loss: 0.0479, Accuracy: 0.9903


### Function to create imbalanced dataset

In [5]:
# Function to create imbalanced dataset
def get_imbalanced_data(dataset, num_sample_per_class, shuffle=False, random_seed=0):
    length = dataset.__len__()
    num_sample_per_class = list(num_sample_per_class)
    selected_list = []
    indices = list(range(0, length))

    for i in range(0, length):
        index = indices[i]
        _, label = dataset.__getitem__(index)
        if num_sample_per_class[label] > 0:
            selected_list.append(index)
            num_sample_per_class[label] -= 1

    return selected_list

# Create imbalanced dataset
num_samples_per_class = [5000, 3000, 2000, 1000, 500, 200, 100, 50, 20, 10]
imbalanced_indices = get_imbalanced_data(train_dataset, num_samples_per_class)
imbalanced_dataset = Subset(train_dataset, imbalanced_indices)
imbalanced_loader = DataLoader(imbalanced_dataset, batch_size=64, shuffle=True)

### applying SMOTE ...

In [6]:
# Extract data and labels for SMOTE
data = []
labels = []
for d, l in imbalanced_loader:
    data.append(d)
    labels.append(l)
data = torch.cat(data).numpy().reshape(-1, 28*28)
labels = torch.cat(labels).numpy()

# Apply SMOTE
smote = SMOTE()
data_resampled, labels_resampled = smote.fit_resample(data, labels)
data_resampled = torch.tensor(data_resampled).reshape(-1, 1, 28, 28).float()
labels_resampled = torch.tensor(labels_resampled)

# Create DataLoader for balanced dataset
balanced_dataset = torch.utils.data.TensorDataset(data_resampled, labels_resampled)
balanced_loader = DataLoader(balanced_dataset, batch_size=64, shuffle=True)

# Train the CNN model on balanced dataset
model_balanced = CNN()
optimizer_balanced = optim.Adam(model_balanced.parameters(), lr=0.001)
train_model(model_balanced, balanced_loader, criterion, optimizer_balanced)
inputs_balanced, logits_balanced, balanced_accuracy = test_model(model_balanced, test_loader)

print(f'Original dataset accuracy: {original_accuracy}')
print(f'Balanced dataset accuracy after SMOTE: {balanced_accuracy}')

Epoch 1/10, Loss: 6.818747351644561e-05
Epoch 2/10, Loss: 0.01915144920349121
Epoch 3/10, Loss: 0.00026585900923237205
Epoch 4/10, Loss: 6.3774700720387045e-06
Epoch 5/10, Loss: 0.007075726054608822
Epoch 6/10, Loss: 1.4117374121269677e-05
Epoch 7/10, Loss: 0.0002755784662440419
Epoch 8/10, Loss: 7.8975983797136e-07
Epoch 9/10, Loss: 5.960463056453591e-08
Epoch 10/10, Loss: 1.3485437193594407e-06
Test set: Average loss: 1.7792, Accuracy: 0.8443
Original dataset accuracy: 0.9903
Balanced dataset accuracy after SMOTE: 0.8443


In [7]:
print(f'Original dataset accuracy: {original_accuracy*100}')
print(f'Balanced dataset accuracy after SMOTE: {balanced_accuracy*100}')

Original dataset accuracy: 99.03
Balanced dataset accuracy after SMOTE: 84.43


In [8]:
# Print input vectors and logits
print("\nInput Vectors and Logits for Original Dataset:")
for i in range(len(inputs_original)):
    print(f"Batch {i+1}:")
    print("Inputs:", inputs_original[i])
    print("Logits:", logits_original[i])

print("\nInput Vectors and Logits for Balanced Dataset:")
for i in range(len(inputs_balanced)):
    print(f"Batch {i+1}:")
    print("Inputs:", inputs_balanced[i])
    print("Logits:", logits_balanced[i])


Input Vectors and Logits for Original Dataset:
Batch 1:
Inputs: tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,

